In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import json
import pandas as pd
import numpy as np
#!pip install pandas_datareader to validate crypto pricing with specific date
import pandas_datareader as web
import datetime as dt
from datetime import date
import seaborn as sns
from numerize import numerize
import torch #torch first before matplotlib, otherwise the library will crash the environment. 
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import string
#NLP libraries
import re
# Import nltk modules and download dataset
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download("omw-1.4")

stop = set(stopwords.words('english'))

#import FinBert library
from textblob import TextBlob
from sklearn.metrics import classification_report


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
#extract data libraries
import requests
import dateutil.parser
import unicodedata
import time
from searchtweets import load_credentials

In [4]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

print(os.getcwd())

C:\Users\eikde\source\repos\help-dissertation


In [5]:
from pprint import pprint
from transformers import AutoModelForSequenceClassification

from finbert import *
import finbert.utils as tools
from finbert.finbert import predict

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd()
print(project_dir)

C:\Users\eikde\source\repos\help-dissertation


In [6]:
#to check whether torch is available. Torch library is important for NLP and BERT.
torch.cuda.is_available()

True

In [7]:
colors = {'red': '#ff207c', 'grey': '#42535b', 'blue': '#207cff', 'orange': '#ffa320', 'green': '#00ec8b'}
config_ticks = {'size': 14, 'color': colors['grey'], 'labelcolor': colors['grey']}
config_title = {'size': 18, 'color': colors['grey'], 'ha': 'left', 'va': 'baseline'}

In [8]:
#plot chart function
def get_charts(data, title):
    plt.rc('figure', figsize=(15, 10))
    fig, axes = plt.subplots(2, 1, 
                gridspec_kw={'height_ratios': [3, 1]})
    fig.tight_layout(pad=3)
    fig.suptitle(title, fontsize=16)
    
    date = data['Date']
    close = data['Close']
    vol = data['Volume']
    
    plot_price = axes[0]
    plot_price.plot(date, close, color=colors['blue'], 
    linewidth=2, label='Price')
    plot_price.set_ylabel('Price (in USD)', fontsize=14)
    plot_price.set_xlabel('Date', fontsize=14)
    
    plot_vol = axes[1]
    plot_vol.bar(date, vol, width=15, color='darkgrey')
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Volume (in millions)', fontsize=14)

In [9]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#a function that to help preprocessing the message to a proper text for analysis
def preprocess_word(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    # Lowercase the message
    try:
        text = str(message).lower()
    except:
        print(text)
    
    
    # Replace % to percentage only fulfill for number
    try:
        replace_percent = re.findall('(\d+(\.\d+)?%)', text)
        for i in range(len(replace_percent)):
            item = re.sub('%', 'percent', replace_percent[i][0])
            percent = item.replace("percent","%")
            item = re.sub('%', ' percent', replace_percent[i][0])
            text = text.replace(percent, item)
    except:
        print(text)
        
    # Replace URLs with a space in the message
    text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    text = re.sub(r'http\S+', '', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
    
    # Replace usernames with a space. The usernames are any word that starts with @.
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub('[^a-z0-9.0-9A-Z.]', ' ', text)
    
    # Remove stop words
    word_tokens = word_tokenize(remove_punctuation(text))
    
    # POS
    pos = nltk.pos_tag(word_tokens)
    
    lemmatizer=WordNetLemmatizer()
    
    text = ""
    for i in pos: 
        if i[1] == "IN" or i[1] =="PRP" or i[1] == "PRP$" or i[1] == "CC":
            text = text + ""
        else:
            #lemmatization
            text = text + lemmatizer.lemmatize(i[0]) + " "
        
    word_tokens = word_tokenize(text)

    
    filtered = []
    filtered = [w for w in word_tokens if not w in stop]
    filtered = TreebankWordDetokenizer().detokenize(filtered)
    
    return filtered

In [10]:
# Check whether the function is working. 
test_message = 'Yields on government bonds fell as investors fled to the traditional haven in a market storm.'
print(preprocess_word(test_message))


yield government bond fell investor fled traditional market storm


In [11]:
#load back the pre-train model
#load back the pre-train model
cl_path = project_dir/'Models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [18]:
#Test statement
text = "world bank cut malaysia 2021 gdp growth projection 45 kuala lumpur june 23 world bank ha slashed gross domestic \
product well better external demand according bank negara malaysia"

In [19]:
result = predict(preprocess_word(text),model, use_gpu=True, gpu_name='cuda:0',batch_size=100)

11/15/2022 22:05:42 - INFO - root -   Using device: cuda:0 
11/15/2022 22:05:42 - INFO - finbert.utils -   *** Example ***
11/15/2022 22:05:42 - INFO - finbert.utils -   guid: 0
11/15/2022 22:05:42 - INFO - finbert.utils -   tokens: [CLS] world bank cut malaysia 2021 gdp growth projection 45 kuala lumpur june 23 world bank ha slashed gross domestic product well better external demand according bank ne ##gara malaysia [SEP]
11/15/2022 22:05:42 - INFO - finbert.utils -   input_ids: 101 2088 2924 3013 6027 25682 14230 3930 13996 3429 15490 17761 2238 2603 2088 2924 5292 23587 7977 4968 4031 2092 2488 6327 5157 2429 2924 11265 24864 6027 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/15/2022 22:05:42 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/15/2022 22:05:42 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [20]:
result

,sentence,logit,prediction,sentiment_score
0,world bank cut malaysia 2021 gdp growth projec...,"[0.019699423, 0.9722679, 0.008032662]",negative,-0.952569


In [45]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is 0.41.


In [23]:
#Test statement
text = "Longer-duration government bonds and corporate bonds in the infrastructure and automotive sectors \
        are two bright spots investors would want to look at. However, equities are expected to outperform \
        fixed income as the global economy continues to recover, says RHB Banking Group chief economist \
        Dr Sailesh Kumar Jha. According to Jha, longer-duration government bonds represent investment opportunities \
        as yields spiked at the end of March while prices fell. For instance, the yield for the 10-year Malaysian \
        government securities (MGS) rose to about 4% on March 28 from about 3.7% a week ago."
result = predict(text,model, use_gpu=True, gpu_name='cuda:0',batch_size=100)
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

11/15/2022 22:18:44 - INFO - root -   Using device: cuda:0 
11/15/2022 22:18:44 - INFO - finbert.utils -   *** Example ***
11/15/2022 22:18:44 - INFO - finbert.utils -   guid: 0
11/15/2022 22:18:44 - INFO - finbert.utils -   tokens: [CLS] longer - duration government bonds and corporate bonds in the infrastructure and automotive sectors are two bright spots investors would want to look at . [SEP]
11/15/2022 22:18:44 - INFO - finbert.utils -   input_ids: 101 2936 1011 9367 2231 9547 1998 5971 9547 1999 1996 6502 1998 12945 11105 2024 2048 4408 7516 9387 2052 2215 2000 2298 2012 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/15/2022 22:18:44 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/15/2022 22:18:44 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Average sentiment is 0.59.


In [24]:
result

,sentence,logit,prediction,sentiment_score
0,Longer-duration government bonds and corporate...,"[0.11288774, 0.005092598, 0.8820196]",neutral,0.107795
1,"However, equities are expected to outperform ...","[0.74240077, 0.24260642, 0.014992902]",positive,0.499794
2,"According to Jha, longer-duration government b...","[0.8505036, 0.030983042, 0.118513376]",positive,0.819521
3,"For instance, the yield for the 10-year Malays...","[0.9609692, 0.02106559, 0.017965214]",positive,0.939904


### Generate finance news sentiment

In [46]:
#read excel file
news_government_policy = pd.read_csv(r"Data/News/key_2021_1_new_data.csv", sep="\t")
news_government_policy.drop(columns = news_government_policy.columns[0], axis = 1, inplace= True)

news_government_policy2 = pd.read_csv(r"Data/News/key_2021_2_new_data.csv", sep="\t")
news_government_policy2.drop(columns = news_government_policy2.columns[0], axis = 1, inplace= True)

news_government_policy3 = pd.read_csv(r"Data/News/key_2021_3_new_data.csv", sep="\t")
news_government_policy3.drop(columns = news_government_policy3.columns[0], axis = 1, inplace= True)

news_government_policy4 = pd.read_csv(r"Data/News/key_2021_4_new_data.csv", sep="\t")
news_government_policy4.drop(columns = news_government_policy4.columns[0], axis = 1, inplace= True)

news_government_policy5 = pd.read_csv(r"Data/News/key_2021_5_new_data.csv", sep="\t")
news_government_policy5.drop(columns = news_government_policy5.columns[0], axis = 1, inplace= True)

In [47]:
news_policy = news_government_policy.append(news_government_policy2, ignore_index=True)
news_policy = news_policy.append(news_government_policy3, ignore_index=True)
news_policy = news_policy.append(news_government_policy4, ignore_index=True)
news_policy = news_policy.append(news_government_policy5, ignore_index=True)

In [48]:
#read excel file
news_key = pd.read_csv(r"Data/News/key1_new_data.csv", sep="\t")
news_key.drop(columns = news_key.columns[0], axis = 1, inplace= True)

#read excel file
news_key2 = pd.read_csv(r"Data/News/key2_new_data.csv", sep="\t")
news_key2.drop(columns = news_key2.columns[0], axis = 1, inplace= True)

#read excel file
news_key3 = pd.read_csv(r"Data/News/key2_new_data.csv", sep="\t")
news_key3.drop(columns = news_key3.columns[0], axis = 1, inplace= True)

#read excel file
news_key4 = pd.read_csv(r"Data/News/key3_new_data.csv", sep="\t")
news_key4.drop(columns = news_key4.columns[0], axis = 1, inplace= True)

In [49]:
news_policy = news_policy.append(news_key, ignore_index=True)
news_policy = news_policy.append(news_key2, ignore_index=True)
news_policy = news_policy.append(news_key3, ignore_index=True)
news_policy = news_policy.append(news_key4, ignore_index=True)

In [50]:
news_policy

,publisher,title,description,created_on,keyword
0,The Edge Markets,No urgency for unconventional monetary policy ...,KUALA LUMPUR (Nov 13): Malaysia sees no pressi...,13 Nov 2020,Malaysia Government Policy
1,Al Jazeera,Malaysia budget: Government plans to maintain ...,“The government believes that the current fisc...,6 Nov 2020,Malaysia Government Policy
2,The Edge Markets,Government to allocate RM322.5b in Budget 2021,"According to the Fiscal Policy Overview, civil...",6 Nov 2020,Malaysia Government Policy
3,CodeBlue,"Government Slashes 2021 Medical, Public Health...","KUALA LUMPUR, Nov 6 — The government has decre...",6 Nov 2020,Malaysia Government Policy
4,Reuters,Malaysia to charge Top Glove over poor worker ...,KUALA LUMPUR (Reuters) - Malaysia said on Tues...,1 Dec 2020,Malaysia Government Policy
...,...,...,...,...,...
3695,The Star,Bank Negara on the alert,KUALA LUMPUR: Bank Negara is closely monitorin...,25 Feb 2020,Bank Negara Malaysia
3696,Astro Awani,Merungkai kekeliruan kadar kemiskinan,Kadar kemiskinan rasmi di Malaysia dikira berd...,26 Aug 2019,Bank Negara Malaysia
3697,The Malaysian Reserve,Foreign selling likely to persist in near...,Bank Islam Malaysia Bhd chief economist Dr Moh...,19 Aug 2019,Bank Negara Malaysia
3698,Sinar Harian,"GrabPay, e-Wallet rakyat percepat ekonomi digi...","Menurutnya, penggunaan pembayaran berasaskan e...",15 Jan 2020,Bank Negara Malaysia


In [51]:
news_sector = pd.read_csv(r"Data/News/sector_new_data.csv", sep="\t")
news_sector.drop(columns = news_sector.columns[0], axis = 1, inplace= True)

news_sector2 = pd.read_csv(r"Data/News/sector_2_new_data.csv", sep="\t")
news_sector2.drop(columns = news_sector2.columns[0], axis = 1, inplace= True)

news_sector3 = pd.read_csv(r"Data/News/sector_3_new_data.csv", sep="\t")
news_sector3.drop(columns = news_sector3.columns[0], axis = 1, inplace= True)

news_sector4 = pd.read_csv(r"Data/News/sector_4_new_data.csv", sep="\t")
news_sector4.drop(columns = news_sector4.columns[0], axis = 1, inplace= True)

news_sector5 = pd.read_csv(r"Data/News/sector_5_new_data.csv", sep="\t")
news_sector5.drop(columns = news_sector5.columns[0], axis = 1, inplace= True)

news_sector6 = pd.read_csv(r"Data/News/sector_6_new_data.csv", sep="\t")
news_sector6.drop(columns = news_sector6.columns[0], axis = 1, inplace= True)

news_sector7 = pd.read_csv(r"Data/News/sector_7_new_data.csv", sep="\t")
news_sector7.drop(columns = news_sector7.columns[0], axis = 1, inplace= True)

news_sector8 = pd.read_csv(r"Data/News/sector_8_new_data.csv", sep="\t")
news_sector8.drop(columns = news_sector8.columns[0], axis = 1, inplace= True)

news_sector9 = pd.read_csv(r"Data/News/sector_9_new_data.csv", sep="\t")
news_sector9.drop(columns = news_sector9.columns[0], axis = 1, inplace= True)

news_sector10 = pd.read_csv(r"Data/News/sector_2021_1_new_data.csv", sep="\t")
news_sector10.drop(columns = news_sector10.columns[0], axis = 1, inplace= True)

news_sector11 = pd.read_csv(r"Data/News/sector_2021_2_new_data.csv", sep="\t")
news_sector11.drop(columns = news_sector11.columns[0], axis = 1, inplace= True)

news_sector12 = pd.read_csv(r"Data/News/sector_2021_3_new_data.csv", sep="\t")
news_sector12.drop(columns = news_sector12.columns[0], axis = 1, inplace= True)

news_sector13 = pd.read_csv(r"Data/News/sector_2021_4_new_data.csv", sep="\t")
news_sector13.drop(columns = news_sector13.columns[0], axis = 1, inplace= True)

news_sector14 = pd.read_csv(r"Data/News/sector_2021_5_new_data.csv", sep="\t")
news_sector14.drop(columns = news_sector14.columns[0], axis = 1, inplace= True)

news_sector15 = pd.read_csv(r"Data/News/sector_2021_6_new_data.csv", sep="\t")
news_sector15.drop(columns = news_sector15.columns[0], axis = 1, inplace= True)

news_sector16 = pd.read_csv(r"Data/News/sector_2021_7_new_data.csv", sep="\t")
news_sector16.drop(columns = news_sector16.columns[0], axis = 1, inplace= True)

In [52]:
news_sector = news_sector.append(news_sector2, ignore_index=True)
news_sector = news_sector.append(news_sector3, ignore_index=True)
news_sector = news_sector.append(news_sector4, ignore_index=True)
news_sector = news_sector.append(news_sector5, ignore_index=True)
news_sector = news_sector.append(news_sector6, ignore_index=True)
news_sector = news_sector.append(news_sector7, ignore_index=True)
news_sector = news_sector.append(news_sector8, ignore_index=True)
news_sector = news_sector.append(news_sector9, ignore_index=True)
news_sector = news_sector.append(news_sector10, ignore_index=True)
news_sector = news_sector.append(news_sector11, ignore_index=True)
news_sector = news_sector.append(news_sector12, ignore_index=True)
news_sector = news_sector.append(news_sector13, ignore_index=True)
news_sector = news_sector.append(news_sector14, ignore_index=True)
news_sector = news_sector.append(news_sector15, ignore_index=True)
news_sector = news_sector.append(news_sector16, ignore_index=True)

In [53]:
news_sector

,publisher,title,description,created_on,keyword
0,The Edge Markets,"KLCI slumps to 11-year low as MAHB, banks and ...",KUALA LUMPUR (March 19): The FBM KLCI fell to ...,19 Mar 2020,KLCI
1,The Edge Markets,KLCI sinks below 1300 mark to its lowest in a ...,KUALA LUMPUR (March 16): The FBM KLCI dropped ...,16 Mar 2020,KLCI
2,The Edge Markets,KLCI down 2.69% as Malaysia stocks enter into ...,KUALA LUMPUR (Feb 24): The FBM KLCI closed dow...,24 Feb 2020,KLCI
3,The Edge Markets,"KLCI reverses loss, marginally higher as Genti...",KUALA LUMPUR (April 2): The FBM KLCI reversed ...,2 Apr 2020,KLCI
4,The Malaysian Reserve,FBM KLCI has gained over 20% since March’s...,by DASHVEENJIT KAUR/ pic by ARIF KARTONO. THE ...,1 Jun 2020,KLCI
...,...,...,...,...,...
4895,Southern Alliance for Clean Energy,"Utility Shut-Offs, Summer Heat, and COVID-19: ...","Utility Shut-Offs, Summer Heat, and COVID-19: ...",25 Aug 2020,Utilities
4896,Power Technology,Cybersecurity in Power: Trends in Utilities,Power utilities' security threats to grow. Uti...,24 Sept 2020,Utilities
4897,The Economic Times,Government plans $41 billion reform to revive ...,The plan for carrying out reforms for reducing...,27 Jan 2021,Utilities
4898,S&P Global,Path to net-zero: US utilities in no rush to m...,Although several U.S. utilities have raised th...,16 Jun 2021,Utilities


In [54]:
news_stock = pd.read_csv(r"Data/News/stock1_new_data.csv", sep="\t")
news_stock.drop(columns = news_stock.columns[0], axis = 1, inplace= True)

news_stock2 = pd.read_csv(r"Data/News/stock2_new_data.csv", sep="\t")
news_stock2.drop(columns = news_stock2.columns[0], axis = 1, inplace= True)

news_stock3 = pd.read_csv(r"Data/News/stock3_new_data.csv", sep="\t")
news_stock3.drop(columns = news_stock3.columns[0], axis = 1, inplace= True)

news_stock4 = pd.read_csv(r"Data/News/stock4_new_data.csv", sep="\t")
news_stock4.drop(columns = news_stock4.columns[0], axis = 1, inplace= True)

news_stock5 = pd.read_csv(r"Data/News/stock5_new_data.csv", sep="\t")
news_stock5.drop(columns = news_stock5.columns[0], axis = 1, inplace= True)

news_stock6 = pd.read_csv(r"Data/News/stock6_new_data.csv", sep="\t")
news_stock6.drop(columns = news_stock6.columns[0], axis = 1, inplace= True)

news_stock7 = pd.read_csv(r"Data/News/stock7_new_data.csv", sep="\t")
news_stock7.drop(columns = news_stock7.columns[0], axis = 1, inplace= True)

news_stock8 = pd.read_csv(r"Data/News/stock8_new_data.csv", sep="\t")
news_stock8.drop(columns = news_stock8.columns[0], axis = 1, inplace= True)

news_stock9 = pd.read_csv(r"Data/News/stock9_new_data.csv", sep="\t")
news_stock9.drop(columns = news_stock9.columns[0], axis = 1, inplace= True)

news_stock10 = pd.read_csv(r"Data/News/stock10_new_data.csv", sep="\t")
news_stock10.drop(columns = news_stock10.columns[0], axis = 1, inplace= True)

news_stock11 = pd.read_csv(r"Data/News/stock11_new_data.csv", sep="\t")
news_stock11.drop(columns = news_stock11.columns[0], axis = 1, inplace= True)

news_stock12 = pd.read_csv(r"Data/News/stock12_new_data.csv", sep="\t")
news_stock12.drop(columns = news_stock12.columns[0], axis = 1, inplace= True)

news_stock13 = pd.read_csv(r"Data/News/stock13_new_data.csv", sep="\t")
news_stock13.drop(columns = news_stock13.columns[0], axis = 1, inplace= True)

news_stock14 = pd.read_csv(r"Data/News/stock14_new_data.csv", sep="\t")
news_stock14.drop(columns = news_stock14.columns[0], axis = 1, inplace= True)

news_stock15 = pd.read_csv(r"Data/News/stock15_new_data.csv", sep="\t")
news_stock15.drop(columns = news_stock15.columns[0], axis = 1, inplace= True)

news_stock16 = pd.read_csv(r"Data/News/stock16_new_data.csv", sep="\t")
news_stock16.drop(columns = news_stock16.columns[0], axis = 1, inplace= True)

news_stock17 = pd.read_csv(r"Data/News/stock17_new_data.csv", sep="\t")
news_stock17.drop(columns = news_stock17.columns[0], axis = 1, inplace= True)

news_stock18 = pd.read_csv(r"Data/News/stock18_new_data.csv", sep="\t")
news_stock18.drop(columns = news_stock18.columns[0], axis = 1, inplace= True)

In [55]:
news_stock_2021 = pd.read_csv(r"Data/News/Stock_2021_1_new_data.csv", sep="\t")
news_stock_2021.drop(columns = news_stock_2021.columns[0], axis = 1, inplace= True)

news_stock_2021_2 = pd.read_csv(r"Data/News/Stock_2021_2_new_data.csv", sep="\t")
news_stock_2021_2.drop(columns = news_stock_2021_2.columns[0], axis = 1, inplace= True)

news_stock_2021_3 = pd.read_csv(r"Data/News/Stock_2021_3_new_data.csv", sep="\t")
news_stock_2021_3.drop(columns = news_stock_2021_3.columns[0], axis = 1, inplace= True)

news_stock_2021_4 = pd.read_csv(r"Data/News/Stock_2021_4_new_data.csv", sep="\t")
news_stock_2021_4.drop(columns = news_stock_2021_4.columns[0], axis = 1, inplace= True)

news_stock_2021_5 = pd.read_csv(r"Data/News/Stock_2021_5_new_data.csv", sep="\t")
news_stock_2021_5.drop(columns = news_stock_2021_5.columns[0], axis = 1, inplace= True)

news_stock_2021_6 = pd.read_csv(r"Data/News/Stock_2021_6_new_data.csv", sep="\t")
news_stock_2021_6.drop(columns = news_stock_2021_6.columns[0], axis = 1, inplace= True)

news_stock_2021_7 = pd.read_csv(r"Data/News/Stock_2021_7_new_data.csv", sep="\t")
news_stock_2021_7.drop(columns = news_stock_2021_7.columns[0], axis = 1, inplace= True)

news_stock_2021_8 = pd.read_csv(r"Data/News/Stock_2021_8_new_data.csv", sep="\t")
news_stock_2021_8.drop(columns = news_stock_2021_8.columns[0], axis = 1, inplace= True)

news_stock_2021_9 = pd.read_csv(r"Data/News/Stock_2021_9_new_data.csv", sep="\t")
news_stock_2021_9.drop(columns = news_stock_2021_9.columns[0], axis = 1, inplace= True)

news_stock_2021_10 = pd.read_csv(r"Data/News/Stock_2021_10_new_data.csv", sep="\t")
news_stock_2021_10.drop(columns = news_stock_2021_10.columns[0], axis = 1, inplace= True)

news_stock_2021_11 = pd.read_csv(r"Data/News/Stock_2021_11_new_data.csv", sep="\t")
news_stock_2021_11.drop(columns = news_stock_2021_11.columns[0], axis = 1, inplace= True)

news_stock_2021_12 = pd.read_csv(r"Data/News/Stock_2021_12_new_data.csv", sep="\t")
news_stock_2021_12.drop(columns = news_stock_2021_12.columns[0], axis = 1, inplace= True)

news_stock_2021_13 = pd.read_csv(r"Data/News/Stock_2021_13_new_data.csv", sep="\t")
news_stock_2021_13.drop(columns = news_stock_2021_13.columns[0], axis = 1, inplace= True)

news_stock_2021_14 = pd.read_csv(r"Data/News/Stock_2021_14_new_data.csv", sep="\t")
news_stock_2021_14.drop(columns = news_stock_2021_14.columns[0], axis = 1, inplace= True)

news_stock_2021_15 = pd.read_csv(r"Data/News/Stock_2021_15_new_data.csv", sep="\t")
news_stock_2021_15.drop(columns = news_stock_2021_15.columns[0], axis = 1, inplace= True)

news_stock_2021_16 = pd.read_csv(r"Data/News/Stock_2021_16_new_data.csv", sep="\t")
news_stock_2021_16.drop(columns = news_stock_2021_16.columns[0], axis = 1, inplace= True)

news_stock_2021_17 = pd.read_csv(r"Data/News/Stock_2021_17_new_data.csv", sep="\t")
news_stock_2021_17.drop(columns = news_stock_2021_17.columns[0], axis = 1, inplace= True)

news_stock_2021_18 = pd.read_csv(r"Data/News/Stock_2021_18_new_data.csv", sep="\t")
news_stock_2021_18.drop(columns = news_stock_2021_18.columns[0], axis = 1, inplace= True)

news_stock_2021_19 = pd.read_csv(r"Data/News/Stock_2021_19_new_data.csv", sep="\t")
news_stock_2021_19.drop(columns = news_stock_2021_19.columns[0], axis = 1, inplace= True)

news_stock_2021_20 = pd.read_csv(r"Data/News/Stock_2021_20_new_data.csv", sep="\t")
news_stock_2021_20.drop(columns = news_stock_2021_20.columns[0], axis = 1, inplace= True)

news_stock_2021_21 = pd.read_csv(r"Data/News/Stock_2021_21_new_data.csv", sep="\t")
news_stock_2021_21.drop(columns = news_stock_2021_21.columns[0], axis = 1, inplace= True)

news_stock_2021_22 = pd.read_csv(r"Data/News/Stock_2021_22_new_data.csv", sep="\t")
news_stock_2021_22.drop(columns = news_stock_2021_22.columns[0], axis = 1, inplace= True)

news_stock_2021_23 = pd.read_csv(r"Data/News/Stock_2021_23_new_data.csv", sep="\t")
news_stock_2021_23.drop(columns = news_stock_2021_23.columns[0], axis = 1, inplace= True)

news_stock_2021_24 = pd.read_csv(r"Data/News/Stock_2021_24_new_data.csv", sep="\t")
news_stock_2021_24.drop(columns = news_stock_2021_24.columns[0], axis = 1, inplace= True)

news_stock_2021_25 = pd.read_csv(r"Data/News/Stock_2021_25_new_data.csv", sep="\t")
news_stock_2021_25.drop(columns = news_stock_2021_25.columns[0], axis = 1, inplace= True)

news_stock_2021_26 = pd.read_csv(r"Data/News/Stock_2021_26_new_data.csv", sep="\t")
news_stock_2021_26.drop(columns = news_stock_2021_26.columns[0], axis = 1, inplace= True)

In [56]:
news_stock = news_stock.append(news_stock2, ignore_index=True)
news_stock = news_stock.append(news_stock3, ignore_index=True)
news_stock = news_stock.append(news_stock4, ignore_index=True)
news_stock = news_stock.append(news_stock5, ignore_index=True)
news_stock = news_stock.append(news_stock6, ignore_index=True)
news_stock = news_stock.append(news_stock7, ignore_index=True)
news_stock = news_stock.append(news_stock8, ignore_index=True)
news_stock = news_stock.append(news_stock9, ignore_index=True)
news_stock = news_stock.append(news_stock10, ignore_index=True)
news_stock = news_stock.append(news_stock11, ignore_index=True)
news_stock = news_stock.append(news_stock12, ignore_index=True)
news_stock = news_stock.append(news_stock13, ignore_index=True)
news_stock = news_stock.append(news_stock14, ignore_index=True)
news_stock = news_stock.append(news_stock15, ignore_index=True)
news_stock = news_stock.append(news_stock16, ignore_index=True)
news_stock = news_stock.append(news_stock17, ignore_index=True)
news_stock = news_stock.append(news_stock18, ignore_index=True)
news_stock = news_stock.append(news_stock_2021, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_2, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_3, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_4, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_5, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_6, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_7, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_8, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_9, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_10, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_11, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_12, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_13, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_14, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_15, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_16, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_17, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_18, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_19, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_20, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_21, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_22, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_23, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_24, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_25, ignore_index=True)
news_stock = news_stock.append(news_stock_2021_26, ignore_index=True)

In [57]:
news_stock

,publisher,title,description,created_on,keyword
0,The Spoon,Court Rules That Nestlé's 'Incredible' Burger ...,A European court ruled that Néstle must change...,1 Jun 2020,Nestle
1,FoodBev Media,Impossible Foods forces Nestlé to rename its “...,"Following the court ruling, Nestlé is prohibit...",2 Jun 2020,Nestle
2,Green Queen,Nestlé Loses “Incredible Burger” Name After Le...,Swiss food giant Nestlé has just lost a legal ...,3 Jun 2020,Nestle
3,Food Processing,Nestlé Shifts Ice Cream to Joint Venture,Nestlé will transfer its North American ice cr...,11 Dec 2019,Nestle
4,Reuters,Nestle sells U.S. ice cream brands to joint ve...,Froneri was created after Nestle merged its Eu...,11 Dec 2019,Nestle
...,...,...,...,...,...
10135,Astro Awani,MAHB lantik Zambry sebagai pengerusi baharu,Malaysia Airports Holdings Berhad (MAHB) melan...,11 Aug 2020,Malaysia Airport
10136,The Malaysian Reserve,Highly anticipated immunisation to start on...,MALAYSIA will begin its national vaccination p...,17 Feb 2021,Malaysia Airport
10137,Focus on Travel News,Malaysia Airports Remains Optimistic with 20 N...,Air travel will be boosted at the network of a...,16 Apr 2021,Malaysia Airport
10138,New Straits Times,MAHB expects positive sales from KLIA Crazy Sale,KUALA LUMPUR: Malaysia Airports Holdings Bhd (...,23 Mar 2021,Malaysia Airport


In [58]:
news =  news_policy.append(news_stock, ignore_index=True)
news =  news.append(news_sector, ignore_index=True)

news.to_csv(r"Data/News/all_news.csv", sep="\t")

In [59]:
news

,publisher,title,description,created_on,keyword
0,The Edge Markets,No urgency for unconventional monetary policy ...,KUALA LUMPUR (Nov 13): Malaysia sees no pressi...,13 Nov 2020,Malaysia Government Policy
1,Al Jazeera,Malaysia budget: Government plans to maintain ...,“The government believes that the current fisc...,6 Nov 2020,Malaysia Government Policy
2,The Edge Markets,Government to allocate RM322.5b in Budget 2021,"According to the Fiscal Policy Overview, civil...",6 Nov 2020,Malaysia Government Policy
3,CodeBlue,"Government Slashes 2021 Medical, Public Health...","KUALA LUMPUR, Nov 6 — The government has decre...",6 Nov 2020,Malaysia Government Policy
4,Reuters,Malaysia to charge Top Glove over poor worker ...,KUALA LUMPUR (Reuters) - Malaysia said on Tues...,1 Dec 2020,Malaysia Government Policy
...,...,...,...,...,...
18735,Southern Alliance for Clean Energy,"Utility Shut-Offs, Summer Heat, and COVID-19: ...","Utility Shut-Offs, Summer Heat, and COVID-19: ...",25 Aug 2020,Utilities
18736,Power Technology,Cybersecurity in Power: Trends in Utilities,Power utilities' security threats to grow. Uti...,24 Sept 2020,Utilities
18737,The Economic Times,Government plans $41 billion reform to revive ...,The plan for carrying out reforms for reducing...,27 Jan 2021,Utilities
18738,S&P Global,Path to net-zero: US utilities in no rush to m...,Although several U.S. utilities have raised th...,16 Jun 2021,Utilities


In [ ]:
sentiment_result = pd.DataFrame()
fieldnames = ['date','category', 'publisher','content', 'sentiment_score']

for index, row in news.iterrows():
    date = row['created_on']
    content = preprocess_word(row['title']) + ' ' + preprocess_word(row['description'])
    category = row['keyword']
    publisher = row['publisher']
    
    predict_content = predict(content, model, use_gpu=True, gpu_name='cuda:0', batch_size=64)
    sentiment_score = predict_content.sentiment_score.mean()
    
    record = [(date, category, publisher, content, sentiment_score)]
    records = pd.DataFrame(record, columns = fieldnames)
    sentiment_result = pd.concat([sentiment_result, records], ignore_index=True, axis=0)

sentiment_result.to_csv(r'Data/sentiment_result/sentiment.csv', sep="\t")

11/15/2022 21:46:28 - INFO - root -   Using device: cuda:0 
11/15/2022 21:46:28 - INFO - finbert.utils -   *** Example ***
11/15/2022 21:46:28 - INFO - finbert.utils -   guid: 0
11/15/2022 21:46:28 - INFO - finbert.utils -   tokens: [CLS] urgency unconventional monetary policy bn ##m kuala lumpur nov 13 malaysia see pressing need employ unconventional monetary policy support government fiscal spending battle [SEP]
11/15/2022 21:46:28 - INFO - finbert.utils -   input_ids: 101 19353 23693 12194 3343 24869 2213 15490 17761 13292 2410 6027 2156 7827 2342 12666 23693 12194 3343 2490 2231 10807 5938 2645 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/15/2022 21:46:28 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/15/2022 21:46:28 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 